In [16]:
import pandas as pd
import difflib

In [17]:
data = pd.read_csv('../Gpt-4/afrisent/gpt4_responses-afrisent.csv')
data.head()

,instruction,input,ouptput,datasource,response
0,"የተሰጠው ጽሑፍ አስተያየት ምን አይነት ነው? ""አዎንታዊ""፣ ""አሉታዊ"" ወ...",'@user ክብር እና ምስጋና ለዓለማት ፈጣሪ ይሁን',አዎንታዊ,afrisent,ይህ የተሰጠው ጽሑፍ ዐዎንታዊ አስተያየት ነው።
1,"የተሰጠው ጽሑፍ አስተያየት ምን አይነት ነው? ""አዎንታዊ""፣ ""አሉታዊ"" ወ...",'ከህወሓት ጋር ድርድር ማለት ኢትዮጲያን ማፍረስ ዕቁብ መጣል ነው። #No...,አሉታዊ,afrisent,ጽሑፉ አሉታዊ አስተያየት ነው።
2,"የተሰጠው ጽሑፍ አስተያየት ምን አይነት ነው? ""አዎንታዊ""፣ ""አሉታዊ"" ወ...",'እግዚአብሔር የተመሰገነ ይሁን ኢትዮጵያ አሸነፈች ። እንኳን ደስአለኝ ።',አዎንታዊ,afrisent,"ይህ ጽሑፍ ""አዎንታዊ"" አስተያየት ነው።"
3,"የተሰጠው ጽሑፍ አስተያየት ምን አይነት ነው? ""አዎንታዊ""፣ ""አሉታዊ"" ወ...",'ይህንን ካዩ እባክዎን ከእኔ ጋር ሶስት ያድርጉ',ገለልተኛ,afrisent,"ይህ አስተያየት ""አሉታዊ"" ነው።"
4,"የተሰጠው ጽሑፍ አስተያየት ምን አይነት ነው? ""አዎንታዊ""፣ ""አሉታዊ"" ወ...",'@user ሲሆን ሲሆን ወንድ ትወልጅያለሽ ሳይሆ ሳይሆን ሴት ትወልጃለሽ ...,አሉታዊ,afrisent,"የተሰጠው ምላሽ ""ገለልተኛ"" ነው።"


In [18]:
def string_similarity(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).ratio()

def isin_metrics(str1,str2):
    return str1 in str2

In [19]:
for index, row in data.iterrows():
    similarity = string_similarity(row['ouptput'], row['response'])


In [20]:
similarity

0.25806451612903225

In [21]:
unique_classes = data.ouptput.unique().tolist()
unique_classes += ["nothing_predicted"]
unique_classes

['አዎንታዊ', 'አሉታዊ', 'ገለልተኛ', 'nothing_predicted']

In [22]:
classes = {class_name:index for index,class_name in enumerate(unique_classes)}
y_true = []
y_pred = []




In [23]:
for index, row in data.iterrows():
    y_true.append(classes[row['ouptput']])
    output = [isin_metrics(str1,row['response']) for str1 in classes.keys()]
    
    if True in output:
        y_pred.append(output.index(True))
    else:
        # TODO : how should we handle in the case where we dont find any of the classes
        y_pred.append(classes['nothing_predicted'])


In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=classes.keys()))

                   precision    recall  f1-score   support

            አዎንታዊ       0.28      0.67      0.40       438
             አሉታዊ       0.76      0.33      0.46      1337
            ገለልተኛ       0.18      0.27      0.22       224
nothing_predicted       0.00      0.00      0.00         0

         accuracy                           0.40      1999
        macro avg       0.31      0.32      0.27      1999
     weighted avg       0.59      0.40      0.42      1999



/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
